In [3]:
%matplotlib inline
import pandas as pd
import numpy as np
import os
# Import the main functionality from the SimFin Python API.
import simfin as sf

# Import names used for easy access to SimFin's data-columns.
from simfin.names import *

import yfinance as yf
import yahoo_fin.stock_info as si
# import pandas_datareader.data as web

from dateutil.relativedelta import relativedelta
from datetime import datetime

### Part.1 Create Target T0_5 and T1_5

In [11]:
#load daily price data and earning dates data
df_dailyprice=pd.read_csv('data/sp500_dailyprice.csv')
df_earningdate=pd.read_csv('data/sp500_earningdate.csv')
# df_dailyprice[df_dailyprice['ticker']=='TSLA']

In [20]:
df_dailyprice.head()

,date,open,high,low,close,T0,volume,ticker
0,2016-01-04,41.060001,41.189999,40.340000,40.689999,38.603630,3287300,A
1,2016-01-05,40.730000,40.950001,40.340000,40.549999,38.470814,2587200,A
2,2016-01-06,40.240002,40.990002,40.049999,40.730000,38.641582,2103600,A
3,2016-01-07,40.139999,40.150002,38.810001,39.000000,37.000282,3504300,A
4,2016-01-08,39.220001,39.709999,38.470001,38.590000,36.611309,3736700,A


In [21]:
df_dailyprice.rename(columns={'adjclose':'T0'},inplace=True)
df_target=df_dailyprice[['date','ticker','T0']]
df_target['T0_pre1']=df_target.groupby('ticker')['T0'].shift(1)
df_target['T1']=df_target.groupby('ticker')['T0'].shift(-1)
df_target['T5']=df_target.groupby('ticker')['T0'].shift(-5)
df_target['T0_5']=(df_target['T5']-df_target['T0'])/df_target['T0']
df_target['T1_5']=(df_target['T5']-df_target['T1'])/df_target['T1']
df_target['T0_fromlastday']=(df_target['T0']-df_target['T0_pre1'])/df_target['T0_pre1']
# df_target=df_target.drop(columns=['T1','T5'])
# df_target.dropna(inplace=True)
# df_target.iloc[1500:1520]

/var/folders/g0/x29rhtxs77n94c0nv9tn68km0000gn/T/ipykernel_89527/3115066698.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_target['T0_pre1']=df_target.groupby('ticker')['T0'].shift(1)
/var/folders/g0/x29rhtxs77n94c0nv9tn68km0000gn/T/ipykernel_89527/3115066698.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_target['T1']=df_target.groupby('ticker')['T0'].shift(-1)
/var/folders/g0/x29rhtxs77n94c0nv9tn68km0000gn/T/ipykernel_89527/3115066698.py:5: SettingWithCopyWarning: 
A value is trying to be s

In [13]:
df_earningdate.head()

,Ticker,Publish Date,Earning Date,epsestimate,epsactual,epssurprisepct
0,A,2016-09-07,2016-08-17,0.47,0.49,4.70
1,A,2016-12-20,2016-11-15,0.52,0.59,13.24
2,A,2017-03-08,2017-02-14,0.49,0.53,8.16
3,A,2017-06-06,2017-05-22,0.48,0.58,19.83
4,A,2017-09-06,2017-08-15,0.52,0.59,13.03


In [22]:
df_target.head()

,date,ticker,T0,T0_pre1,T1,T5,T0_5,T1_5,T0_fromlastday
0,2016-01-04,A,38.603630,NaN,38.470814,35.994637,-0.067584,-0.064365,NaN
1,2016-01-05,A,38.470814,38.603630,38.641582,36.231827,-0.058200,-0.062362,-0.003441
2,2016-01-06,A,38.641582,38.470814,37.000282,34.970013,-0.095016,-0.054872,0.004439
3,2016-01-07,A,37.000282,38.641582,36.611309,35.681553,-0.035641,-0.025395,-0.042475
4,2016-01-08,A,36.611309,37.000282,35.994637,35.207188,-0.038352,-0.021877,-0.010513


In [15]:
df_target2=pd.merge(df_earningdate[['Ticker','Earning Date']]
                   , df_target
                   , how='left'
                   , left_on=['Ticker','Earning Date']
                   , right_on=['ticker','date'])
# df_target.drop(columns=['ticker','date'],inplace=True)

In [23]:
df_target2.head()

,Ticker,Earning Date,date,ticker,T0,T0_pre1,T0_5,T1_5,T0_fromlastday
0,A,2016-08-17,2016-08-17,A,46.201164,46.058052,-0.035722,-0.000428,0.003107
1,A,2016-11-15,2016-11-15,A,43.418201,43.351246,-0.020705,-0.037246,0.001544
2,A,2017-02-14,2017-02-14,A,48.329895,48.454594,0.018853,-0.005424,-0.002574
3,A,2017-05-22,2017-05-22,A,53.920761,53.968842,0.076498,0.029151,-0.000891
4,A,2017-08-15,2017-08-15,A,57.220871,57.452133,0.060290,0.013196,-0.004025


In [ ]:
# df_target.to_csv("data/target.csv",index=False)

### Part.2 Create Technical indicators

In [13]:
#load daily price data and earning dates data
df_dailyprice=pd.read_csv('data/sp500_dailyprice.csv')
df_earningdate=pd.read_csv('data/sp500_earningdate.csv')

In [4]:
# Simple Moving Average 
def SMA(data, ndays): 
    SMA = pd.Series(data['adjclose'].rolling(ndays).mean(), name = 'SMA_' + str(ndays)) 
    data = data.join(SMA) 
    return data

# Exponentially-weighted Moving Average 
def EWMA(data, ndays): 
    EMA = pd.Series(data['close'].ewm(span = ndays, min_periods = ndays - 1).mean(), 
                 name = 'EWMA_' + str(ndays)) 
    data = data.join(EMA) 
    return data

In [5]:
# Compute the Bollinger Bands 
def BBANDS(data, window):
    MA = data.adjclose.rolling(window).mean()
    SD = data.adjclose.rolling(window).std()
    data['MiddleBand'] = MA
    data['UpperBand'] = MA + (2 * SD) 
    data['LowerBand'] = MA - (2 * SD)
    return data

In [6]:
# Returns RSI values
def rsi(data, periods = 14):
    
    close_delta = data['adjclose'].diff()

    # Make two series: one for lower closes and one for higher closes
    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    
    ma_up = up.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
    ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()

    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))
    data['RSI']=rsi
    return data

In [7]:
# Calculate money flow index
def gain(x):
    return ((x > 0) * x).sum()
def loss(x):
    return ((x < 0) * x).sum()
def mfi(high, low, close, volume, n=14):
    typical_price = (high + low + close)/3
    money_flow = typical_price * volume
    mf_sign = np.where(typical_price > typical_price.shift(1), 1, -1)
    signed_mf = money_flow * mf_sign
    mf_avg_gain = signed_mf.rolling(n).apply(gain, raw=True)
    mf_avg_loss = signed_mf.rolling(n).apply(loss, raw=True)
    return (100 - (100 / (1 + (mf_avg_gain / abs(mf_avg_loss))))).to_numpy()

In [8]:
def createratio(data, column1, column2): 
    ratio = pd.Series(data[column1]/data[column2], name = column1+'_'+column2) 
    data = data.join(ratio) 
    return data

In [14]:
df_dailyprice=SMA(df_dailyprice,12)
df_dailyprice=SMA(df_dailyprice,30)
df_dailyprice=EWMA(df_dailyprice,12)
df_dailyprice=EWMA(df_dailyprice,30)
df_dailyprice=BBANDS(df_dailyprice,30)
df_dailyprice=rsi(df_dailyprice)
df_dailyprice['MFI']=mfi(df_dailyprice['high'],df_dailyprice['low'],df_dailyprice['adjclose'],df_dailyprice['volume'])

In [15]:
for column in ['SMA_12','SMA_30','EWMA_12','EWMA_30','UpperBand','LowerBand']:
    df_dailyprice=createratio(df_dailyprice,'adjclose',column)

df_dailyprice=createratio(df_dailyprice,'SMA_12','SMA_30')
df_dailyprice=createratio(df_dailyprice,'EWMA_12','EWMA_30')
df_dailyprice=createratio(df_dailyprice,'UpperBand','LowerBand')

In [16]:
df_dailyprice.drop(columns=['open','MiddleBand','high', 'low', 'close', 'volume','adjclose','SMA_12','SMA_30','EWMA_12','EWMA_30','UpperBand','LowerBand'],inplace=True)

In [18]:
df_techind=pd.merge(df_earningdate[['Ticker','Earning Date']]
                   , df_dailyprice
                   , how='left'
                   , left_on=['Ticker','Earning Date']
                   , right_on=['ticker','date'])
df_techind.drop(columns=['ticker','date'],inplace=True)
df_techind.dropna(inplace=True)

In [19]:
df_techind.to_csv("data/techind.csv",index=False)